In [ ]:
# Configura conjuntos de treinamento e teste 
# X_train, X_test, Y_train, Y_test
%run './functions/SLD-LoadTrainTestSets.ipynb'

In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Model, Input
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, GlobalAveragePooling2D, Reshape
from keras.optimizers import Adam

In [ ]:
X_train = np.array(convert_to_rgb_npy_images(X_train))
X_test = np.array(convert_to_rgb_npy_images(X_test))

In [ ]:
print('Training shape:', X_train.shape)
print('Test shape:', X_test.shape)

In [ ]:
input_shape = (64, 64, 3)
weights = None
include_top = True

adam_lr = 0.0001
nb_epochs = 10
cm_plot_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
validation_split = 0.1
batch_size=10

---

In [ ]:
base_model = MobileNet(input_shape=input_shape,
                       weights=weights,
                       include_top=include_top)
base_model.summary()

---

In [ ]:
y = base_model.layers[-4].output
y = Conv2D(10, kernel_size=(1, 1), activation='relu')(y)
y = Activation('softmax')(y)
y = Reshape((10,))(y)

In [ ]:
model = Model(inputs=base_model.input, outputs=y)
model.summary()

In [ ]:
len(model.layers)

In [ ]:
# Congela camadas
# for layer in model.layers:
#     layer.trainable = False

In [ ]:
%%time
acertos, pred = execute_model(model, X_train, Y_train, X_test, Y_test,
                              validation_split, batch_size, cm_plot_labels, 
                              adam_lr, nb_epochs)

---

In [ ]:
X_train_resized = np.array(resize_npy_images(X_train, (128, 128)))
X_test_resized = np.array(resize_npy_images(X_test, (128, 128)))

In [ ]:
print('Training shape:', X_train_resized.shape)
print('Test shape:', X_test_resized.shape)

In [ ]:
input_shape = (128, 128, 3)
weights = 'imagenet'

In [ ]:
base_model = MobileNet(input_shape=input_shape,
                       weights=weights,
                       include_top=include_top)
base_model.summary()

In [ ]:
x = MobileNet(input_shape=input_shape,
                       weights=weights,
                       include_top=include_top)
y = base_model.layers[-4].output
y = Conv2D(10, kernel_size=(1, 1), activation='relu')(y)
y = Activation('softmax')(y)
y = Reshape((10,))(y)

In [ ]:
model = Model(inputs=base_model.input, outputs=y)
model.layers[0].trainable = True
model.summary()

In [ ]:
for layer in model.layers:
    print(layer.trainable)

In [ ]:
len(model.layers)

In [ ]:
%%time
acertos, pred = execute_model(model, X_train_resized, Y_train, X_test_resized, Y_test,
                              validation_split, batch_size, cm_plot_labels, 
                              adam_lr, nb_epochs)

In [ ]:
for layer in model.layers:
    print(layer.trainable)